In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling /chisel-bootcamp/Main.sc

Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml


Compiling /chisel-bootcamp/Main.sc #2

path: String = "/chisel-bootcamp/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

import dotvisualizer._

In [4]:
class AES128 extends Module {
  val io = IO(new Bundle{
    val plaintext = Input(UInt(128.W))
    val key       = Input(UInt(128.W))
    val start     = Input(Bool())
    val done      = Output(Bool())
    val ciphertext= Output(UInt(128.W))
  })

  private def byte(i: UInt, idx: Int) = {
    (i >> (8*idx)).asUInt()(7,0)
  }

  private val sboxBytes = Seq(
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
  ).map(_.toInt)

  private val sbox = VecInit(sboxBytes.map(b => b.U(8.W)))

  private val rcon = Seq(0x01,0x02,0x04,0x08,0x10,0x20,0x40,0x80,0x1B,0x36).map(_.toInt)

  private def mul2(b: UInt) = {
    val top = b(7)
    val shifted = (b << 1)(7,0)
    Mux(top === 1.U, shifted ^ 0x1b.U(8.W), shifted)
  }
  private def mul3(b: UInt) = mul2(b) ^ b
    
  private def toState(w: UInt): Vec[Vec[UInt]] = {
    val cols = Wire(Vec(4, Vec(4, UInt(8.W))))
    for (c <- 0 until 4) {
      for (r <- 0 until 4) {
        cols(c)(r) := byte(w, (c*4 + r))
      }
    }
    cols
  }

  private def fromState(cols: Vec[Vec[UInt]]): UInt = {
    var out = 0.U(128.W)
    for (c <- 0 until 4) {
      for (r <- 0 until 4) {
        out = out | (cols(c)(r).asUInt << (8*(c*4 + r))).asUInt
      }
    }
    out
  }

  // SubBytes
  private def subBytes(cols: Vec[Vec[UInt]]): Vec[Vec[UInt]] = {
    val out = Wire(Vec(4, Vec(4, UInt(8.W))))
    for (c <- 0 until 4) for (r <- 0 until 4) out(c)(r) := sbox(cols(c)(r))
    out
  }

  // ShiftRows
  private def shiftRows(cols: Vec[Vec[UInt]]): Vec[Vec[UInt]] = {
    val out = Wire(Vec(4, Vec(4, UInt(8.W))))
    for (c <- 0 until 4) for (r <- 0 until 4) out(c)(r) := cols((c + r) % 4)(r)
    out
  }

  // MixColumns on a column
  private def mixColumn(col: Vec[UInt]): Vec[UInt] = {
    val a0 = col(0); val a1 = col(1); val a2 = col(2); val a3 = col(3)
    val r0 = (mul2(a0) ^ mul3(a1) ^ a2 ^ a3)
    val r1 = (a0 ^ mul2(a1) ^ mul3(a2) ^ a3)
    val r2 = (a0 ^ a1 ^ mul2(a2) ^ mul3(a3))
    val r3 = (mul3(a0) ^ a1 ^ a2 ^ mul2(a3))
    VecInit(r0, r1, r2, r3)
  }

  private def mixColumns(cols: Vec[Vec[UInt]]): Vec[Vec[UInt]] = {
    val out = Wire(Vec(4, Vec(4, UInt(8.W))))
    for (c <- 0 until 4) out(c) := mixColumn(cols(c))
    out
  }

  // Key expansion
  private def expandKey(key: UInt): Vec[UInt] = {
    val w = Wire(Vec(44, UInt(32.W)))
    for (i <- 0 until 4) {
      w(i) := (key >> (32*(3-i))).asUInt()(31,0)
    }
    for (i <- 4 until 44) {
      var temp = w(i-1)
      if (i % 4 == 0) {
        val rot = Cat(temp(23,0), temp(31,24))
        val sub = Wire(UInt(32.W))
        sub := Cat(sbox(rot(31,24)), sbox(rot(23,16)), sbox(rot(15,8)), sbox(rot(7,0)))
        temp = (sub ^ (rcon((i/4)-1).U(32.W) << 24))
      }
      w(i) := w(i-4) ^ temp
    }
    val rk = Wire(Vec(11, UInt(128.W)))
    for (r <- 0 until 11) {
      var k = 0.U(128.W)
      for (j <- 0 until 4) {
        k = k | (w(r*4 + j).asUInt << (32*(3-j)))
      }
      rk(r) := k
    }
    rk
  }

  // one round: SubBytes, ShiftRows, MixColumns, AddRoundKey
  private def round(state: UInt, roundKey: UInt, finalRound: Boolean = false): UInt = {
    val st = toState(state)
    val sb = subBytes(st)
    val sr = shiftRows(sb)
    val mc = if (finalRound) sr else mixColumns(sr)
    fromState(mc) ^ roundKey
  }

  // --- control FSM ---
  val idle :: busy :: done :: Nil = Enum(3)
  val stateReg = RegInit(idle)
  val roundCnt = RegInit(0.U(4.W))
  val stateRegData = Reg(UInt(128.W))
  val roundKeys = expandKey(io.key)

  io.done := false.B
  io.ciphertext := 0.U

  switch(stateReg) {
    is(idle) {
      when(io.start) {
        stateReg := busy
        roundCnt := 0.U
        stateRegData := io.plaintext ^ roundKeys(0)
      }
    }
    is(busy) {
      when(roundCnt === 10.U) {
        // final round
        stateRegData := round(stateRegData, roundKeys(10), true)
        stateReg := done
      }.otherwise {
        stateRegData := round(stateRegData, roundKeys(roundCnt + 1.U))
        roundCnt := roundCnt + 1.U
      }
    }
    is(done) {
      io.done := true.B
      io.ciphertext := stateRegData
      when(!io.start) { stateReg := idle }
    }
  }
}


defined class AES128

In [7]:
test(new AES128) { dut =>
  val key = BigInt("000102030405060708090a0b0c0d0e0f", 16)
  val pt  = BigInt("00112233445566778899aabbccddeeff", 16)
  val expected = BigInt("18b270f2547cdcea0a21990b6d23c5b0", 16)

  dut.io.key.poke(key.U)
  dut.io.plaintext.poke(pt.U)
  dut.io.start.poke(true.B)
  dut.clock.step(1)
  dut.io.start.poke(false.B)

  while (dut.io.done.peek().litToBoolean == false) {
    dut.clock.step(1)
  }

  val out = dut.io.ciphertext.peek().litValue
  println(f"Ciphertext: $out%032x")
  assert(out == expected, "AES output does not match expected value!")
}


Elaborating design...
Done elaborating.
Ciphertext: 18b270f2547cdcea0a21990b6d23c5b0
test AES128 Success: 0 tests passed in 14 cycles in 0.516623 seconds 27.10 Hz
